<a href="https://colab.research.google.com/github/shamim237/python-dev-task/blob/sub-main/Python_Dev_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install required libraries**

In [ ]:
!pip install transformers
!pip install sentencepiece

**Import required libraries**

In [ ]:
import re
import torch
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import PegasusForConditionalGeneration, AutoTokenizer

**scrape data from the Amazon webpage using scrapperAPI and BeautifulSoup**

In [ ]:
url = "https://www.amazon.com/dp/B09B9TB61G?th=1"
params = {
    "api_key": "3c259788e88e8921b5dab304e9913bd3",
    "url": url
}
response = requests.get("http://api.scraperapi.com", params=params)

soup = BeautifulSoup(response.content, 'html.parser')
product_description = soup.find('div', attrs={'id': 'aplus'})

In [ ]:
filt = []
global prod_description
print('\033[92mRAW DATA\033[0m')
print('='*200)
for i in product_description.find_all("p"):
  sen = i.text.strip()
  print(sen)
  filt.append(sen)
prod_description = list(filter(None, filt))
prod_description = [sentence.strip() for sentence in prod_description]

RAW DATA
EIGIIS military smart watch has a 10 military grade certifications and can be used under the harsh environmental conditions, such as 70 ℃ heat resistance, -40 ℃ cold resistance; 120 hours resistance to humidity, 96 hours resistance to salt spray.
EIGIIS military smart watch has a variety of exercise modes, including running, walking, cycling, skipping, basketball, football, badminton, etc. It can perform real-time heart rate monitoring and accurately record all-day activities, such as pedometer, distance, calories burned, heart rate and active time through intelligent exercise algorithms.
After connecting to Bluetooth, you can make calls on your EIGIIS smart watch K22, and you can view various notifications such as SMS and SNS (Facebook, WhatsApp, Instagram, Wechat and Twitter ) through your smart watch without turning on your phone.



You can make calls and answer the calls directly by your watch. Besides, it can also supports call rejection and synchronize your recent calls

**Paraphrasing model from HuggingFace🤗**

In [ ]:
def paraphrase_model(model_name):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return model

def para_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer

model = paraphrase_model("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = para_tokenizer("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def para(paragraph):
    paras = []
    for sentence in paragraph:
      text = "paraphrase: " + sentence + " </s>"
      encoding = tokenizer.encode_plus(text,max_length = 256, padding=True, return_tensors="pt")
      input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
      model.eval()
      beam_outputs = model.generate(
          input_ids=input_ids,attention_mask=attention_mask,
          max_length= 256,
          early_stopping=True,
          num_beams=15,
          num_return_sequences=3)
      sent = tokenizer.decode(beam_outputs[2], skip_special_tokens=True,clean_up_tokenization_spaces=True)
      paras.append(sent)

    parast = []
    for i in paras:
        resf = i.replace("paraphrasedoutput: ", "")
        parast.append(resf)
    return parast

**Text Summarization model from HuggingFace🤗**

In [ ]:
def load_model(model_name):
    model = PegasusForConditionalGeneration.from_pretrained(model_name)
    return model

def load_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer

model = load_model("google/pegasus-cnn_dailymail")
tokenizer = load_tokenizer("google/pegasus-cnn_dailymail")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def summarize():
    sentence = para(prod_description)
    txt = "".join(sentence)
    txt = txt.replace("\n", " ").replace("<n>", " ")
    batch = tokenizer(txt, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    summy = tokenizer.batch_decode(translated, skip_special_tokens=True)
    summy = ' '.join(summy)
    summary = summy.replace('<n>', ' ')
    summary = summary.replace(". ", ".\n")
    return summary

**The below Script will produce the Summary of the scrapted data from Amazon website.**

In [ ]:
summary = summarize()
print("\033[95mSummary:\033[0m")
print("="*200)
print("\033[91m {}\033[00m" .format(summary))


Summary:
 EIGIIS military smart watch has a variety of exercise modes, including running, walking, cycling, skipping, basketball, football, badminton, etc.
It can perform real-time heart rate monitoring and accurately record all-day activities through intelligent exercise algorithms.
Using message and call notification, you will never miss any important calls and messages at all.
